In [1]:
# docs and experiment results can be found at https://docs.cleanrl.dev/rl-algorithms/ppo/#ppo_pettingzoo_ma_ataripy
import random
import time

import gymnasium as gym
import numpy as np
import supersuit as ss
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter

from pettingzoo.atari import flag_capture_v2

In [2]:
SEED = 1
TORCH_DETERMINISTIC = True
CAPTURE_VIDEO = False

In [3]:
HYPERPARAMETERS = {
    'n_timesteps': 50_000,  # total timesteps of the experiments
    'learning_rate': 2.5e-4,  # the learning rate of the optimizer
    'n_envs': 16,  # the number of parallel game environments
    'n_steps': 128,  # the number of steps to run in each environment per policy rollout
    'anneal_lr': True,  # toggle learning rate annealing for policy and value networks
    'gamma': 0.99,  # the discount factor gamma
    'gae_lambda': 0.95,  # the lambda for the general advantage estimation
    'n_minibatches': 4,  # the number of mini-batches
    'update_epochs': 4,  # the K epochs to update the policy
    'norm_adv': True,  # Toggles advantages normalization
    'clip_coef': 0.1,  # the surrogate clipping coefficient
    'clip_vloss': True,  # Toggles whether to use a clipped loss for the value function, as per the paper
    'ent_coef': 0.01,  # coefficient of the entropy
    'vf_coef': 0.5,  # coefficient of the value function
    'max_grad_norm': 0.5,  # the maximum norm for the gradient clipping
    'target_kl': None,  # the target KL divergence threshold (float)

    'batch_size': 16 * 128,
    'minibatch_size': (16 * 128) // 4
}

In [4]:
def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

In [5]:
class Agent(nn.Module):
    def __init__(self, envs):
        super().__init__()
        self.network = nn.Sequential(
            layer_init(nn.Conv2d(6, 32, 8, stride=4)),
            nn.ReLU(),
            layer_init(nn.Conv2d(32, 64, 4, stride=2)),
            nn.ReLU(),
            layer_init(nn.Conv2d(64, 64, 3, stride=1)),
            nn.ReLU(),
            nn.Flatten(),
            layer_init(nn.Linear(64 * 7 * 7, 512)),
            nn.ReLU(),
        )
        self.actor = layer_init(nn.Linear(512, envs.single_action_space.n), std=0.01)
        self.critic = layer_init(nn.Linear(512, 1), std=1)

    def get_value(self, x):
        x = x.clone()
        x[:, :, :, [0, 1, 2, 3]] /= 255.0
        return self.critic(self.network(x.permute((0, 3, 1, 2))))

    def get_action_and_value(self, x, action=None):
        x = x.clone()
        x[:, :, :, [0, 1, 2, 3]] /= 255.0
        hidden = self.network(x.permute((0, 3, 1, 2)))
        logits = self.actor(hidden)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(hidden)

In [6]:
if __name__ == "__main__":
    run_name = f"flag_capture_v2__ppo_pettingzoo_ma_atari__{SEED}__{int(time.time())}"

    writer = SummaryWriter(f"runs/{run_name}")
    writer.add_text(
        "hyperparameters",
        "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in HYPERPARAMETERS.items()])),
    )

    # TRY NOT TO MODIFY: seeding
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.backends.cudnn.deterministic = TORCH_DETERMINISTIC

    device = torch.device("cpu")

    # env setup
    env = flag_capture_v2.parallel_env()
    env = ss.max_observation_v0(env, 2)
    env = ss.frame_skip_v0(env, 4)
    env = ss.clip_reward_v0(env, lower_bound=-1, upper_bound=1)
    env = ss.color_reduction_v0(env, mode="B")
    env = ss.resize_v1(env, x_size=84, y_size=84)
    env = ss.frame_stack_v1(env, 4)
    env = ss.agent_indicator_v0(env, type_only=False)
    env = ss.pettingzoo_env_to_vec_env_v1(env)
    envs = ss.concat_vec_envs_v1(env, HYPERPARAMETERS['n_envs'] // 2, num_cpus=0, base_class="gymnasium")
    envs.single_observation_space = envs.observation_space
    envs.single_action_space = envs.action_space
    envs.is_vector_env = True
    envs = gym.wrappers.RecordEpisodeStatistics(envs)
    if CAPTURE_VIDEO:
        envs = gym.wrappers.RecordVideo(envs, f"videos/{run_name}")
    assert isinstance(envs.single_action_space, gym.spaces.Discrete), "only discrete action space is supported"

    agent = Agent(envs).to(device)
    optimizer = optim.Adam(agent.parameters(), lr=HYPERPARAMETERS['learning_rate'], eps=1e-5)

    # ALGO Logic: Storage setup
    obs = torch.zeros((HYPERPARAMETERS['n_steps'], HYPERPARAMETERS['n_envs']) + envs.single_observation_space.shape).to(device)
    actions = torch.zeros((HYPERPARAMETERS['n_steps'], HYPERPARAMETERS['n_envs']) + envs.single_action_space.shape).to(device)
    logprobs = torch.zeros((HYPERPARAMETERS['n_steps'], HYPERPARAMETERS['n_envs'])).to(device)
    rewards = torch.zeros((HYPERPARAMETERS['n_steps'], HYPERPARAMETERS['n_envs'])).to(device)
    dones = torch.zeros((HYPERPARAMETERS['n_steps'], HYPERPARAMETERS['n_envs'])).to(device)
    values = torch.zeros((HYPERPARAMETERS['n_steps'], HYPERPARAMETERS['n_envs'])).to(device)

    # TRY NOT TO MODIFY: start the game
    global_step = 0
    start_time = time.time()
    next_obs = torch.Tensor(envs.reset()).to(device)
    next_done = torch.zeros(HYPERPARAMETERS['n_envs']).to(device)
    num_updates = HYPERPARAMETERS['n_timesteps'] // HYPERPARAMETERS['batch_size']

    for update in range(1, num_updates + 1):
        # Annealing the rate if instructed to do so.
        if HYPERPARAMETERS['anneal_lr']:
            frac = 1.0 - (update - 1.0) / num_updates
            lrnow = frac * HYPERPARAMETERS['learning_rate']
            optimizer.param_groups[0]["lr"] = lrnow

        for step in range(0, HYPERPARAMETERS['n_steps']):
            global_step += 1 * HYPERPARAMETERS['n_envs']
            obs[step] = next_obs
            dones[step] = next_done

            # ALGO LOGIC: action logic
            with torch.no_grad():
                action, logprob, _, value = agent.get_action_and_value(next_obs)
                values[step] = value.flatten()
            actions[step] = action
            logprobs[step] = logprob

            # TRY NOT TO MODIFY: execute the game and log data.
            next_obs, reward, done, info = envs.step(action.cpu().numpy())
            rewards[step] = torch.tensor(reward).to(device).view(-1)
            next_obs, next_done = torch.Tensor(next_obs).to(device), torch.Tensor(done).to(device)

            for idx, item in enumerate(info):
                player_idx = idx % 2
                if "episode" in item.keys():
                    print(f"global_step={global_step}, {player_idx}-episodic_return={item['episode']['r']}")
                    writer.add_scalar(f"charts/episodic_return-player{player_idx}", item["episode"]["r"], global_step)
                    writer.add_scalar(f"charts/episodic_length-player{player_idx}", item["episode"]["l"], global_step)

        # bootstrap value if not done
        with torch.no_grad():
            next_value = agent.get_value(next_obs).reshape(1, -1)
            advantages = torch.zeros_like(rewards).to(device)
            lastgaelam = 0
            for t in reversed(range(HYPERPARAMETERS['n_steps'])):
                if t == HYPERPARAMETERS['n_steps'] - 1:
                    nextnonterminal = 1.0 - next_done
                    nextvalues = next_value
                else:
                    nextnonterminal = 1.0 - dones[t + 1]
                    nextvalues = values[t + 1]
                delta = rewards[t] + HYPERPARAMETERS['gamma'] * nextvalues * nextnonterminal - values[t]
                advantages[t] = lastgaelam = delta + HYPERPARAMETERS['gamma'] * HYPERPARAMETERS['gae_lambda'] * nextnonterminal * lastgaelam
            returns = advantages + values

        # flatten the batch
        b_obs = obs.reshape((-1,) + envs.single_observation_space.shape)
        b_logprobs = logprobs.reshape(-1)
        b_actions = actions.reshape((-1,) + envs.single_action_space.shape)
        b_advantages = advantages.reshape(-1)
        b_returns = returns.reshape(-1)
        b_values = values.reshape(-1)

        # Optimizing the policy and value network
        b_inds = np.arange(HYPERPARAMETERS['batch_size'])
        clipfracs = []
        for epoch in range(HYPERPARAMETERS['update_epochs']):
            np.random.shuffle(b_inds)
            for start in range(0, HYPERPARAMETERS['batch_size'], HYPERPARAMETERS['minibatch_size']):
                end = start + HYPERPARAMETERS['minibatch_size']
                mb_inds = b_inds[start:end]

                _, newlogprob, entropy, newvalue = agent.get_action_and_value(b_obs[mb_inds], b_actions.long()[mb_inds])
                logratio = newlogprob - b_logprobs[mb_inds]
                ratio = logratio.exp()

                with torch.no_grad():
                    # calculate approx_kl http://joschu.net/blog/kl-approx.html
                    old_approx_kl = (-logratio).mean()
                    approx_kl = ((ratio - 1) - logratio).mean()
                    clipfracs += [((ratio - 1.0).abs() > HYPERPARAMETERS['clip_coef']).float().mean().item()]

                mb_advantages = b_advantages[mb_inds]
                if HYPERPARAMETERS['norm_adv']:
                    mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)

                # Policy loss
                pg_loss1 = -mb_advantages * ratio
                pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - HYPERPARAMETERS['clip_coef'], 1 + HYPERPARAMETERS['clip_coef'])
                pg_loss = torch.max(pg_loss1, pg_loss2).mean()

                # Value loss
                newvalue = newvalue.view(-1)
                if HYPERPARAMETERS['clip_vloss']:
                    v_loss_unclipped = (newvalue - b_returns[mb_inds]) ** 2
                    v_clipped = b_values[mb_inds] + torch.clamp(
                        newvalue - b_values[mb_inds],
                        -HYPERPARAMETERS['clip_coef'],
                        HYPERPARAMETERS['clip_coef'],
                    )
                    v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                    v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                    v_loss = 0.5 * v_loss_max.mean()
                else:
                    v_loss = 0.5 * ((newvalue - b_returns[mb_inds]) ** 2).mean()

                entropy_loss = entropy.mean()
                loss = pg_loss - HYPERPARAMETERS['ent_coef'] * entropy_loss + v_loss * HYPERPARAMETERS['vf_coef']

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(agent.parameters(), HYPERPARAMETERS['max_grad_norm'])
                optimizer.step()

            if HYPERPARAMETERS['target_kl'] is not None:
                if approx_kl > HYPERPARAMETERS['target_kl']:
                    break

        y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
        var_y = np.var(y_true)
        explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

        # TRY NOT TO MODIFY: record rewards for plotting purposes
        writer.add_scalar("charts/learning_rate", optimizer.param_groups[0]["lr"], global_step)
        writer.add_scalar("losses/value_loss", v_loss.item(), global_step)
        writer.add_scalar("losses/policy_loss", pg_loss.item(), global_step)
        writer.add_scalar("losses/entropy", entropy_loss.item(), global_step)
        writer.add_scalar("losses/old_approx_kl", old_approx_kl.item(), global_step)
        writer.add_scalar("losses/approx_kl", approx_kl.item(), global_step)
        writer.add_scalar("losses/clipfrac", np.mean(clipfracs), global_step)
        writer.add_scalar("losses/explained_variance", explained_var, global_step)
        print("SPS:", int(global_step / (time.time() - start_time)))
        writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)

    envs.close()
    writer.close()

/Users/milosz/anaconda3/envs/CI-in-DDA/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.single_action_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.single_action_space` for environment variables or `env.get_wrapper_attr('single_action_space')` that will search the reminding wrappers.
  logger.warn(
/Users/milosz/anaconda3/envs/CI-in-DDA/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.single_observation_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.single_observation_space` for environment variables or `env.get_wrapper_attr('single_observation_space')` that will search the reminding wrappers.
  logger.warn(
/Users/milosz/anaconda3/envs/CI-in-DDA/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.single_action_space to get variables from oth

(array([[[[  0,   0,   0,   0, 255,   0],
         [  0,   0,   0,   0, 255,   0],
         [  0,   0,   0,   0, 255,   0],
         ...,
         [  0,   0,   0,   0, 255,   0],
         [  0,   0,   0,   0, 255,   0],
         [  0,   0,   0,   0, 255,   0]],

        [[  0,   0,   0,   0, 255,   0],
         [  0,   0,   0,   0, 255,   0],
         [  0,   0,   0,   0, 255,   0],
         ...,
         [  0,   0,   0,   0, 255,   0],
         [  0,   0,   0,   0, 255,   0],
         [  0,   0,   0,   0, 255,   0]],

        [[  0,   0,   0, 175, 255,   0],
         [  0,   0,   0, 175, 255,   0],
         [  0,   0,   0, 175, 255,   0],
         ...,
         [  0,   0,   0, 175, 255,   0],
         [  0,   0,   0, 175, 255,   0],
         [  0,   0,   0, 175, 255,   0]],

        ...,

        [[  0,   0,   0, 176, 255,   0],
         [  0,   0,   0, 176, 255,   0],
         [  0,   0,   0, 176, 255,   0],
         ...,
         [  0,   0,   0, 176, 255,   0],
         [  0,   0,  

ValueError: expected sequence of length 84 at dim 2 (got 0)